In [ ]:
%tensorflow_version 1
#!pip install tensorflow==1.13.1
!pip install -q gpt-2-simple

In [ ]:
!git clone https://github.com/aquadzn/gpt2-french.git

In [ ]:
#!pip install tensorflow-gpu==1.15.2  # Pour utiliser tensorflow.contrib, il faut installer la version de tensorflow 1.15.2
#!pip install tensorflow-estimatior==2.1.0
#from tensorflow.python.compiler.tensorrt import trt_convert as trt
import gpt_2_simple as gpt2
import glob
import nltk

from datetime import datetime
from google.colab import files

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
gpt2.download_gpt2(model_name="355M")

In [ ]:
gpt2.download_gpt2(model_name="124M")

In [ ]:
import os
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)  

In [ ]:
gpt2.mount_gdrive()

# Conte de grimms en français

In [ ]:
import glob

In [ ]:
grimms = sorted(glob.glob('/content/drive/My Drive/projet-TIA/dataset/Conte-de-grimms/*'))


In [ ]:
import re
def clean_text(text):
  pattern = '(page|Page|PAGE)(\s+\|\s+)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned


In [ ]:
text=""
for book in grimms:
  with open(book) as f:
    print('Reading', book.split('/')[-1])
    temp = f.read()
    text = text + clean_text(temp)

In [ ]:
with open("/content/drive/My Drive/Cleaned_grimms_french.txt",'w') as f:
     f.writelines(text)

In [ ]:
grimms_clean = "Cleaned_grimms_french.txt"

In [ ]:
gpt2.copy_file_from_gdrive(grimms_clean)

# Entrainement du modèle

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=grimms_clean,
              model_name='124M',
              steps=3000,
              restore_from='fresh',
              checkpoint_dir = "drive/My Drive/projet-TIA/gpt2/checkpoint/",
              run_name='run_grimms_french-3000',
              sample_every=100,
              save_every=500
              )

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_french-3000')

# Générer du texte

In [ ]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.reset_session(sess)
del sess
sess = gpt2.start_tf_sess()

In [ ]:
%%time
gpt2.load_gpt2(sess, run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_french-3000')

In [ ]:
gpt2.generate(sess,
              run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_french-3000',
              length=50, 
              top_k=40,
              top_p=1.0,
              temperature=0.7,
              prefix="Dans des temps très anciens, alors qu'il pouvait encore être utile de faire des vœux, vivait un roi dont toutes les filles étaient belles."
              )

# Conte d'Andersen en français

In [ ]:
andersen = sorted(glob.glob('/content/drive/My Drive/projet-TIA/dataset/andersen/*'))


In [ ]:
import re
def clean_text(text):
  pattern = '(page|Page|PAGE)(\s+\|\s+)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned


In [ ]:
text=""
for book in andersen:
  with open(book) as f:
    print('Reading', book.split('/')[-1])
    temp = f.read()
    text = text + clean_text(temp)

In [ ]:
with open("/content/drive/My Drive/Cleaned_andersen_french.txt",'w') as f:
     f.writelines(text)

In [ ]:
andersen_clean = "Cleaned_andersen_french.txt"

In [ ]:
gpt2.copy_file_from_gdrive(andersen_clean)

# Entrainement du modèle

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=andersen_clean,
              model_name=model_name,
              steps=3000,
              restore_from='fresh',
              checkpoint_dir = "drive/My Drive/projet-TIA/gpt2/checkpoint/",
              run_name='run_andersen_french_3000',
              sample_every=100,
              save_every=500
              )

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_french_3000')

# Génerer du texte

In [ ]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.reset_session(sess)
del sess
sess = gpt2.start_tf_sess()

In [ ]:
%%time
gpt2.load_gpt2(sess, run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_french_3000')

Loading checkpoint /content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_french_3000/model-3000
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_french_3000/model-3000
CPU times: user 2.61 s, sys: 846 ms, total: 3.46 s
Wall time: 6.63 s


In [ ]:
gpt2.generate(sess,
              run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_french_3000',
              length=20, 
              top_k=40,
              top_p=0.9,
              temperature=1.0,
              prefix="Il était une fois un prince qui voulait épouser une princesse, mais une vraie princesse. Il fit le tour de la terre pour en trouver une mais il y avait toujours quelque chose qui clochait ; des princesses, il n'en manquait pas, mais étaient-elles de vraies princesses ? C'était difficile à apprécier, toujours une chose ou l'autre ne lui semblait pas parfaite. Il rentra chez lui tout triste, il aurait tant voulu avoir une véritable princesse."
              )

#Grimms en anglais

In [ ]:
import glob

In [ ]:
grimms_en = sorted(glob.glob('/content/drive/My Drive/projet-TIA/dataset/Grimms_anglais/*'))

In [ ]:
import re
def clean_text(text):
  pattern = '(page|Page|PAGE)(\s+\|\s+)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned


In [ ]:
text=""
for book in books:
  with open(book) as f:
    print('Reading', book.split('/')[-1])
    temp = f.read()
    text = text + clean_text(temp)

In [ ]:
with open("/content/drive/My Drive/Cleaned_grimms_en.txt",'w') as f:
     f.writelines(text)

In [ ]:
grimms_en_clean = "Cleaned_grimms_en.txt"

In [ ]:
gpt2.copy_file_from_gdrive(grimms_en_clean)

# Entrainement du modèle

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=grimms_en_clean,
              model_name='124M',
              steps=3000,
              restore_from='fresh',
              checkpoint_dir = "drive/My Drive/projet-TIA/gpt2/checkpoint/",
              run_name='run_grimms_en-3000',
              sample_every=100,
              save_every=500
              )

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_en-3000')

# Générer du texte

In [ ]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.reset_session(sess)
del sess
sess = gpt2.start_tf_sess()

In [ ]:
%%time
gpt2.load_gpt2(sess, run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_en-3000')

In [ ]:
gpt2.generate(sess,
              run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_grimms_en-3000/',
              length=100, 
              top_k=40,
              top_p=0.9,
              temperature=1.0,
              prefix="Once upon a time there was a princess with golden hair"
              )

#Andersen en anglais

In [ ]:
import glob

In [ ]:
andersen_en = sorted(glob.glob('/content/drive/My Drive/projet-TIA/dataset/Andersen_anglais/*'))

In [ ]:
import re
def clean_text(text):
  pattern = '(page|Page|PAGE)(\s+\|\s+)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned


In [ ]:
text=""
for book in andersen_en:
  with open(book) as f:
    print('Reading', book.split('/')[-1])
    temp = f.read()
    text = text + clean_text(temp)

In [ ]:
with open("/content/drive/My Drive/Cleaned_andersen_en.txt",'w') as f:
     f.writelines(text)

In [ ]:
andersen_en_clean = "Cleaned_andersen_en.txt"

In [ ]:
gpt2.copy_file_from_gdrive(andersen_en_clean)

# Entrainement du modèle

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=grimms_en_clean,
              model_name='124M',
              steps=3000,
              restore_from='fresh',
              checkpoint_dir = "drive/My Drive/projet-TIA/gpt2/checkpoint/",
              run_name='run_andersen_en-3000',
              sample_every=100,
              save_every=500
              )

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_en-3000')

# Générer du texte

In [ ]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.reset_session(sess)
del sess
sess = gpt2.start_tf_sess()

In [ ]:
%%time
gpt2.load_gpt2(sess, run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_en-3000')

In [ ]:
gpt2.generate(sess,
              run_name='/content/drive/MyDrive/projet-TIA/gpt2/checkpoint/run_andersen_en-3000/',
              length=100, 
              top_k=40,
              top_p=0.9,
              temperature=1.0,
              prefix="Once upon a time there was a princess with golden hair"
              )